# Interpretation of [BertForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification) using [**Captum**](https://captum.ai/)

Source code info:

Used notebook: https://gist.github.com/davidefiocco/3e1a0ed030792230a33c726c61f6b3a5 (adaptation to classification from original tutorial on question answering: https://captum.ai/tutorials/Bert_SQUAD_Interpret)

(Used notebook is based on this github issue: https://github.com/pytorch/captum/issues/303)

Related github issue: https://github.com/pytorch/captum/issues/249

---

Used model: [armheb/DNA_bert_6](https://huggingface.co/armheb/DNA_bert_6?text=The+goal+of+life+is+%5BMASK%5D.)



## Load initial libraries, models, data:

In [1]:
!pip install transformers datasets tokenizers --quiet

In [2]:
!pip install captum

In [3]:
TOKENIZER = 'armheb/DNA_bert_6'
HF_ORIG_MODEL_NAME = 'armheb/DNA_bert_6'
HF_MODEL_NAME = 'roa7n/DNABert_K6_G_quad'
DATASET = 'roa7n/G_quad_DNA_tokenized_K6'

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification

from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients

import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

### Load tokenizer:

In [5]:
tokenizer = BertTokenizer.from_pretrained(TOKENIZER, num_labels=2)
tokenizer

PreTrainedTokenizer(name_or_path='armheb/DNA_bert_6', vocab_size=4101, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

#### Custom K-mer tokenization:

In [6]:
def kmers(s, k=6):
  return [s[i:i + k] for i in range(0, len(s)-k+1)]

def tokenization(x): 
  return tokenizer(" ".join(kmers(x["seq"])))

example = {'seq': 'ATGGAAAGAGGCACCATTCT'}
print(f'Example: {example}')

example_kmers = " ".join(kmers(example['seq']))
print(f'Example_kmers: {example_kmers}')

tokenized_example = tokenization(example)
print(f'Tokenization example: {tokenized_example}')

decoded_example = tokenizer.decode(tokenized_example['input_ids'])
print(f'Decoded tokenized example: {decoded_example}')

Example: {'seq': 'ATGGAAAGAGGCACCATTCT'}
Example_kmers: ATGGAA TGGAAA GGAAAG GAAAGA AAAGAG AAGAGG AGAGGC GAGGCA AGGCAC GGCACC GCACCA CACCAT ACCATT CCATTC CATTCT
Tokenization example: {'input_ids': [2, 501, 1989, 3848, 3089, 56, 212, 835, 3325, 999, 3983, 3629, 2214, 650, 2587, 2142, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Decoded tokenized example: [CLS] ATGGAA TGGAAA GGAAAG GAAAGA AAAGAG AAGAGG AGAGGC GAGGCA AGGCAC GGCACC GCACCA CACCAT ACCATT CCATTC CATTCT [SEP]


### Load model:

In [7]:
model = BertForSequenceClassification.from_pretrained(HF_ORIG_MODEL_NAME)
model.to(device)

Some weights of the model checkpoint at armheb/DNA_bert_6 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_6 and are n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

## Get the data:

In [8]:
from datasets import load_dataset

dataset = load_dataset(DATASET)
dataset

Using custom data configuration roa7n--G_quad_DNA_tokenized_K6-226a4e1c505f34ab
Reusing dataset parquet (/home/jovyan/.cache/huggingface/datasets/roa7n___parquet/roa7n--G_quad_DNA_tokenized_K6-226a4e1c505f34ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['seq', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 300000
    })
    test: Dataset({
        features: ['seq', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100000
    })
})

In [9]:
dataset['train'][0]['seq']

'TCATAGACCCGGTCTCATATGACAAGGAGGGGCATGTCAGACAGTACCGAAGAAAACTGATCCTTTCCCTTTAGGGGGTCGGGGAACACGAAGGACCCACTCTGCTACGGGGTGGGACGAAGTCGGGGGGGAGGTACTCGACATGGGTGATAGGTTGGTCAGGGTTACCCTACTTGAACCATGGAGTCTACCTTTACGTC'

In [10]:
print(dataset['train'][0]['input_ids'])

[2, 1560, 2129, 311, 1231, 815, 3248, 692, 2754, 2811, 3038, 3947, 3485, 1638, 2441, 1558, 2124, 289, 1143, 461, 1829, 3208, 532, 2113, 248, 980, 3908, 3332, 1027, 4093, 4070, 3980, 3618, 2171, 477, 1896, 3473, 1591, 2253, 808, 3218, 569, 2263, 847, 3376, 1201, 693, 2760, 2833, 3125, 197, 773, 3079, 14, 44, 161, 630, 2507, 1823, 3182, 426, 1690, 2651, 2399, 1391, 1454, 1706, 2714, 2649, 2392, 1364, 1348, 1284, 1028, 4098, 4091, 4064, 3956, 3524, 1796, 3073, 4085, 4039, 3853, 3111, 144, 561, 2229, 712, 2836, 3137, 247, 975, 3887, 3245, 679, 2702, 2603, 2206, 620, 2467, 1662, 2537, 1943, 3664, 2356, 1220, 772, 3074, 4092, 4068, 3972, 3585, 2039, 4048, 3889, 3253, 712, 2834, 3131, 224, 884, 3524, 1796, 3076, 4100, 4100, 4097, 4088, 4052, 3906, 3321, 983, 3918, 3371, 1184, 625, 2487, 1741, 2854, 3212, 548, 2180, 514, 2044, 4065, 3958, 3529, 1816, 3156, 322, 1274, 988, 3940, 3458, 1531, 2013, 3944, 3476, 1604, 2306, 1018, 4057, 3927, 3407, 1327, 1198, 681, 2711, 2638, 2346, 1180, 609, 2421,

## Captum interpretation:

### 1. On untrained model:

*Helper function to perform forward pass of the model and make predictions:*

In [11]:
def predict(inputs):
    score = model(inputs)[0]
    return score

*Custom forward function that will allow us to access the postitions of our prediction using position input argument:*

In [12]:
### original for question answering looked like this: ######
# def squad_pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
#     pred = predict(inputs,
#                    token_type_ids=token_type_ids,
#                    position_ids=position_ids,
#                    attention_mask=attention_mask)
#     pred = pred[position]
#     return pred.max(1).values
############################################################

def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0][0].unsqueeze(-1)

Compute attributions with respect to the `BertEmbeddings` layer:

1. define baselines/ references,
2. numericalize baselines and inputs.

*(helper functions to achieve that)*

In [13]:
ref_token_id = tokenizer.pad_token_id  # token used for generating token reference
sep_token_id = tokenizer.sep_token_id  # token added to the end of the input text
cls_token_id = tokenizer.cls_token_id  # token used at the beginning of the input text
print(ref_token_id)
print(sep_token_id)
print(cls_token_id)

0
3
2


In [14]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

Define the input sequence `sample1` that we'd like to use as an input for our Bert model and interpret what the model was focusing on when predicting the class:

In [15]:
def get_sample_and_label(dataset, seq_id):
    seq = dataset[seq_id]['seq']
    label = dataset[seq_id]['label']
    seq_kmers = kmers(seq)
    tokenized_seq = tokenizer.encode(seq_kmers, add_special_tokens=False)
    print(f'Seq label: {label}')
    print(f'Input seq (raw): {seq}')
    print(f'Returned (tokenized[:10]): {tokenized_seq[:10]}')
    return tokenized_seq, label

sample1, sample_label1 = get_sample_and_label(dataset['test'], 0)

Seq label: 0
Input seq (raw): NNNNNNNNNNNATGTTTATTTTGACTATTTACCACTGTTCTTGGTGTTTTTATGGAGAAAAATGTTTTCAGAGGTTTTTATTCTGCCAGTTGGAGGTACTTCTCTGTAACTGACTTCCGTAGTTAAGTGAAGGACTTATTCCCAAAAACCTGATTTTAGCTCAGGTTTTATAAATTNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Let's numericalize the input `sample1` and generate corresponding baselines/references for all three sub-embeddings (word, token type and position embeddings) types using our helper functions defined above:

In [16]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(sample1, ref_token_id, sep_token_id, cls_token_id)
print(input_ids)
print(ref_input_ids)
print(sep_id)

tensor([[   2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          474, 1881, 3414, 1354, 1306, 1114,  348, 1377, 1399, 1486, 1833, 3222,
          586, 2330, 1113,  343, 1359, 1325, 1191,  654, 2604, 2210,  634, 2523,
         1886, 3434, 1436, 1636, 2434, 1532, 2018, 3962, 3546, 1882, 3418, 1369,
         1366, 1356, 1316, 1153,  504, 2001, 3893, 3269,  773, 3077,    6,   12,
           34,  122,  474, 1882, 3419, 1373, 1384, 1425, 1592, 2260,  834, 3322,
          986, 3930, 3418, 1369, 1366, 1354, 1307, 1118,  364, 1443, 1663, 2541,
         1960, 3730, 2618, 2268,  868, 3457, 1528, 2004, 3906, 3321,  983, 3918,
         3370, 1179,  606, 2411, 1438, 1644, 2466, 1657, 2517, 1863, 3342, 1068,
          161,  631, 2510, 1834, 3227,  607, 2416, 1458, 1721, 2776, 2898, 3386,
         1241,  853, 3400, 1298, 1084,  225,  885, 3528, 1812, 3137,  247,  974,
         3882, 3225,  598, 2378, 1307, 1119,  367, 1453, 1701, 2693, 2565, 2053,
            7,   15,   46,  

In [17]:
model(input_ids)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3168, -0.0104]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [18]:
predict(input_ids)

tensor([[-0.3168, -0.0104]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [19]:
custom_forward(input_ids)

tensor([0.4240], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

There are 2 different ways of computing the attributions for emebdding layers. One option is to use `LayerIntegratedGradients` and compute the attributions with respect to `BertEmbedding`. The second option is to use `LayerIntegratedGradients` for each `word_embeddings`, `token_type_embeddings` and `position_embeddings` and compute the attributions w.r.t each embedding vector.

In [20]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

Helper function to summarize attributions for each word token in the sequence:

In [21]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [22]:
def interpret_and_visualize(tokenized_sample, label):
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(tokenized_sample, ref_token_id, sep_token_id, cls_token_id)
    score = predict(input_ids)

    attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True)
    
    indices = input_ids[0].detach().tolist()
    
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    
    attributions_sum = summarize_attributions(attributions)

    score_vis = viz.VisualizationDataRecord(
        word_attributions=attributions_sum, 
        pred_prob=torch.softmax(score, dim=1)[0][0],
        pred_class=torch.argmax(torch.softmax(score, dim=1)[0]), 
        true_class=label, 
        attr_class=tokenized_sample, 
        attr_score=attributions_sum.sum(), 
        raw_input_ids=all_tokens, 
        convergence_score=delta)

    print('\033[1m', 'Visualization For Score', '\033[0m')
    viz.visualize_text([score_vis])

In [23]:
interpret_and_visualize(sample1, sample_label1)

 Visualization For Score 


Confusion matrix:

In [29]:
# https://stackoverflow.com/questions/68691450/how-can-i-check-a-confusion-matrix-after-fine-tuning-with-custom-datasets

import torch
import torch.nn.functional as F
from sklearn import metrics
from datetime import datetime

def calculate_y_stats(data_samples):
    y_preds = []
    y_trues = []

    print('Start at ' + datetime.now().strftime("%H:%M:%S") + '.')
    
    for i in range(len(data_samples)):
        if i % 10000 == 0:
            print(f' Calculated for {i}.')
            
        sample = data_samples[i]
        tokenized_sample = sample['input_ids']
        input_ids = torch.tensor([tokenized_sample], device=device)
        logits = predict(input_ids)
        prediction = F.softmax(logits, dim=1)
        y_pred = torch.argmax(prediction.cpu()).numpy()
        y_true = sample['label']
        y_preds.append(y_pred)
        y_trues.append(y_true)
    
    print('Done at ' + datetime.now().strftime("%H:%M:%S") + '.')  
    
    return y_preds, y_trues

In [30]:
y_pred1, y_true1  = calculate_y_stats(dataset['test'])

Start at 12:04:32.
 Calculated for 0.
 Calculated for 10000.
 Calculated for 20000.
 Calculated for 30000.
 Calculated for 40000.
 Calculated for 50000.
 Calculated for 60000.
 Calculated for 70000.
 Calculated for 80000.
 Calculated for 90000.
Done at 12:25:00.


In [31]:
metrics.confusion_matrix(y_true1, y_pred1, labels=[1, 0])

array([[28061, 21939],
       [23100, 26900]])

In [32]:
metrics.accuracy_score(y_true1, y_pred1)

0.54961

### 2. On trained model:

In [58]:
trained_model = BertForSequenceClassification.from_pretrained(HF_MODEL_NAME)
trained_model.to(device)

Downloading:   0%|          | 0.00/357M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

### Interpret again

In [59]:
lig = LayerIntegratedGradients(custom_forward, trained_model.bert.embeddings)

In [60]:
def predict(inputs):
    score = trained_model(inputs)[0]
    return score

Confusion matrix

In [61]:
y_pred2, y_true2  = calculate_y_stats(dataset['test'])

Start at 07:59:21.
 Calculated for 0.
 Calculated for 10000.
 Calculated for 20000.
 Calculated for 30000.
 Calculated for 40000.
 Calculated for 50000.
 Calculated for 60000.
 Calculated for 70000.
 Calculated for 80000.
 Calculated for 90000.
Done at 08:11:54.


In [62]:
metrics.confusion_matrix(y_true2, y_pred2, labels=[1, 0])

# array([[48983,  1017],
#        [ 1356, 48644]])

array([[48698,  1302],
       [ 1338, 48662]])

In [63]:
metrics.accuracy_score(y_true2, y_pred2)

0.9736

In [64]:
interpret_and_visualize(sample1, sample_label1)

 Visualization For Score 


## Other input sequences:

### Test data:

In [65]:
for i in range(20):
    sample, sample_label = get_sample_and_label(dataset['test'], i)
    interpret_and_visualize(sample, sample_label)

Seq label: 0
Input seq (raw): NNNNNNNNNNNATGTTTATTTTGACTATTTACCACTGTTCTTGGTGTTTTTATGGAGAAAAATGTTTTCAGAGGTTTTTATTCTGCCAGTTGGAGGTACTTCTCTGTAACTGACTTCCGTAGTTAAGTGAAGGACTTATTCCCAAAAACCTGATTTTAGCTCAGGTTTTATAAATTNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 1
Input seq (raw): AACCCGGGTCCCCTGGGTCCGGGGTGGGGTCGGTTAAGGTAGTGGTTCAAGAAACTGGAGGTGAGGACTGTAACTTGGTCTAGGGACGAGAAGGAGGACGATAGGGGTAACAGGGGACAGGGTCGGTGAGAGTCGTGTATGGACCCAGTAATCTGTCTAGGACAGAGGGCGGAGAGGACGGAGGGGACGACCGGAACAAT
Returned (tokenized[:10]): [176, 692, 2756, 2818, 3067, 4063, 3951, 3503, 1710, 2732]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNNTTGTTCTCTTCTTAGTTCCTTGAGGAGCAACATTATGTTATTTGAGCTCTTTCTTCTTTTTGATATAGGCATTTATTGCTGCAAACATTGATCTTACTGCTTTTGCTGTATTTCATAAGCTTTTGGTATGTTGTGTTTCCATTTTTGTTTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1499, 1886, 3435, 1438, 1642, 2459]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNNNNNNNNNNNNNNNNNNNNNNNNNTGTGGAGGTACAGATGAGAGTGAGGTGGGCATGGGATCAAGCCCTGTAGGGTTCTGGCATTTAGAGGTAGAAGAGAGGAGGACAGGGTGGCAGGTGGAGGAGGCCCAGGGGGGATGGATCCCTGGAACCCAGAGCAGTTCCAGCTGCTAANNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNCATGTACCTTATGCATTCAAATTGATGACTTGCTGACTTGTGTCAATGTTCTATTTTGGAAATACTTGCAAAATTGCGCTTATGATCTTGTATTCCAATGATTTACTTACATCACTATCTCCATCCCATGACTGTGTGCCCTGGAGAGCANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 2169, 471, 1871, 3374, 1194, 665, 2646]
 Visualization For Score 


...

Seq label: 0
Input seq (raw): CTACTGAAAAAGACAATTGATTTCTCCTCTCATTCCCTTGGAAAAGTCAGTTTTTATCCATTTTAAAGTATATTTTAGCATTCCTGATGGTATGTGTATATGCACACATATGTATAGATATATAAATATATACATGTTTTAATTCTCCTCTGAATTGGTTGCAACATCTTATTGGTCATCTCATTAGTGAGTTCAGTAAA
Returned (tokenized[:10]): [2348, 1185, 629, 2501, 1797, 3077, 8, 17, 55, 205]
 Visualization For Score 


Seq label: 0
Input seq (raw): CAAGAATGAAGGATGCTTAGCAGGTGTTGAAGAATCCAAACTTAGAATTGATGCTTCTTAAAGAGGGCTTCTGGGAGCTTAGATCAAGTGGCCATTATCTGGCCCAGCCACCAGCCAATGGGGCTGGAAAGATGGAATGAAAGTTCTGGATTCCCAATCTCTTTGTACTCATCTATATTATTGCACTAATAAAGCTTATT
Returned (tokenized[:10]): [2101, 198, 780, 3105, 117, 456, 1812, 3137, 246, 972]
 Visualization For Score 


Seq label: 0
Input seq (raw): TCTAAAGTTAAGGCTTCAAAACTACATTCCTGAAATTCAAGTTGGCTCAAACAATATCCAACCTCATTTTGCGCAGTCAAAGAAAACTGCTGTTCTTTACTATTGGCTAAAGACATCAAGCCACCACCCTCCTCCAATCTTCAGAGCAATTCATTTGATTCTCAAAATTAGACCTCTGCAAATTTTAAAATTATTTATCA
Returned (tokenized[:10]): [1605, 2312, 1042, 58, 217, 853, 3400, 1300, 1091, 254]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNTTCTAAACTTTGCAAAGTTTTTTTCTTTTAAAAGATTGTTCAATTCTTACCATTGACCTGTGTATATGCCATATCAAGCTATTTAATGACTAGACTTTCATATTTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): CTGAATTGGGTTGAATGGTGGCCTCCCAAAACATATATCTATTTGGAACCTGTAATGGTAACCTTATTTGGAGAAAGGATCTTAGCAAATGTAATTAAGAATCCCAAGATGACATCATCCTGGATAATCCTGGTGAATCCTAAATCCAGTTGCAAGTATCCTTATAAAAGAAAGTTCAGGGCTATATTAGACACATAGAC
Returned (tokenized[:10]): [2502, 1802, 3100, 100, 388, 1538, 2042, 4060, 3937, 3445]
 Visualization For Score 


### Train data:

In [66]:
for i in range(20):
    sample, sample_label = get_sample_and_label(dataset['train'], i)
    interpret_and_visualize(sample, sample_label)

Seq label: 1
Input seq (raw): TCATAGACCCGGTCTCATATGACAAGGAGGGGCATGTCAGACAGTACCGAAGAAAACTGATCCTTTCCCTTTAGGGGGTCGGGGAACACGAAGGACCCACTCTGCTACGGGGTGGGACGAAGTCGGGGGGGAGGTACTCGACATGGGTGATAGGTTGGTCAGGGTTACCCTACTTGAACCATGGAGTCTACCTTTACGTC
Returned (tokenized[:10]): [1560, 2129, 311, 1231, 815, 3248, 692, 2754, 2811, 3038]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNACGAGATCACACAGGTTCTCCGTCGTACACCTCAGTTTTTTCTAATAAGCCGGCCCGCGCCACCGAGTGCGGACATTAGGGTCGTGAAACCCTCCGGCTCCGCCCGCCTAGTGCTCCAGTCCTCTAGCTCTGGTAGGGCCGATTTTGCCACTTTGGGGCATAGATGATTTTTATGTTTTTTAATCGGCCCGCATCN
Returned (tokenized[:10]): [1, 1, 1, 1, 721, 2870, 3275, 797, 3175, 397]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTCCTACCAGAGTTACAGGACTGGAGCACTAGACGGGCGGAGCCGGAGAGTTTCACGACCCTAATGTCCGCACTCGGTGGTGGGGGCCGGGGTCCTGTGTGTCGAAATTTTAAAGAGGAACCAGAGTGGGTCACGGTTGGTGGATTTTGGNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): TTTGAATATAAAAAAGTACCTAGAAGAATAAAAGGCACATTCCTTTATGTATCTTATTAGGCTTTCTTTTTTTCCTAAGTAGTAAAACAATCATTAGTTTCACTGAGTATTTTGGACAGCATATTAGAAATTTGAAGTACCAAGTTTACTATTCTTTACTGGAGATTGTCCAGCAGAGGAAAGCAAATACCAGTGGATGT
Returned (tokenized[:10]): [1397, 1478, 1801, 3094, 73, 277, 1093, 261, 1029, 5]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNNNNNNNNNNNNNGCCTCCCAAACTGCTGGGATTACAGGCGTGAGCCACTGCGCCCGACTGGAAGGGGGCTTTCTTTAGAAAATAGGTAAATGTTAGTAGGGTGGAAACCAACCCTGGCCCCTTTCAAATGTGTGGTTTTTTTTTTTTTTTTTGGAGACAGTTGTACTCTTTTCTCCCNNNNNNNNNNNNNNNNNNNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


...

Seq label: 1
Input seq (raw): CGATTAATAATTCCGCTCTCACCGTCAAGATACAAGGGAGGGGGGGTTTTTTTCCTTCCTTTTCTCCTTTTGATTCAAAGTCACGATTTTATAGAACACCGGGACCCATACGTCCTAACGTATAGACTGGTATTGATCGGATCGTCGCTCAAGGGATGCCTATCGATTTTATGTAGTAAAACTCCACCTAATCGATGGGG
Returned (tokenized[:10]): [2841, 3157, 326, 1289, 1045, 70, 266, 1051, 95, 368]
 Visualization For Score 


Seq label: 1
Input seq (raw): CCAAACTTAGCGCCTTAAGTCAAACACCATTTTATTATATCTCACAATGTTGCATGTCAGGAATTTGGGGCAGGACCCAGCTGGGAGATGCTTTGCAATAGGATGGGGCCCTGGGGGGTGAGTAGGGAGGGGGCTGTGTGTGTGTCGAGGTCATTTGACAATAAAGGCCCTTTTTGCTCTAGTATTCTAAGATTCTTCAA
Returned (tokenized[:10]): [2567, 2062, 42, 153, 600, 2387, 1344, 1267, 959, 3822]
 Visualization For Score 


Seq label: 1
Input seq (raw): NNNNNNNNNNNNNNNNNNAATATGCCATGCATAGATGGAAAAATATCGCATGATCTCACTTATATGTGAAATCTAAAAAAAACAAAAGGCCGAATTGAGGAGACAGAAAATAAAACAGTGGTTACCAGAATTGGGGTGGGGGTGGGAGGAAATGGGGTGACATAGACCAAAGGATACACAGTAGCAAATACATAGGACNN
Returned (tokenized[:10]): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 Visualization For Score 


Seq label: 0
Input seq (raw): GGCCTTCTCTGCATCTATTGAGATAATCATGTGGTTTTTGTCTTTGGCTCTGTTTATATGCTGGATTACACTTATTGATTTGCCTATATTGAACCAGCCTTGCATCCCAGGGATGAAGCCCACTTGATCATGGTGGATAAGCTTTTTGATGTGCTGCTGGATTCGTTTTGCCAGTATTTTATTGAGGATTTTTGCATCAA
Returned (tokenized[:10]): [4010, 3739, 2654, 2411, 1438, 1644, 2467, 1661, 2534, 1931]
 Visualization For Score 


Seq label: 0
Input seq (raw): NNNCTAGAGCAGAGAATTTTAAGACGGTGATCTCTCCTGCTTTCAGTTCCATTTTGGCACAAGTCTTATGTCAACATCTCACCTCCCTTTGATGCAATGATATACACTTTTTCCAATTTAAGAAGGAGTCTGGACTCTGTGGTGTCCTTTCCTTGGTCTATCAAGATCATCCCTTAGGAAGCAGAGCTGAATTCTGGANN
Returned (tokenized[:10]): [1, 1, 1, 2360, 1235, 829, 3304, 913, 3640, 2257]
 Visualization For Score 


*Colours: https://github.com/pytorch/captum/issues/249#issuecomment-580569266*

*That output is the prediction probability (p) of being a positive sentiment. A negative sentiment would be (1 - p).
In our case we attribute positive sentiment probability (p) to the inputs of our model and in case something is predicted with high probability as positive sentiment we see many tokens that contribute positively to the positive sentiment.*

*In case when p is very low, there are no words contributing to the positive sentiment and when we attribute to the positive sentiment prob (p) we find words that pull away (influence negatively) to the positive sentiment. Those tokens are obviously the ones that pull towards the negative sentiment with higher (1-p) probability.*

https://github.com/pytorch/captum/issues/249#issuecomment-580846266

*In a general case, red means that those tokens are pulling away from the Hate Speech (1) and most probably pulling towards the opposite class however I think that red might not always mean that it will always attribute to the other class. I think that's the assumption that we make here. We assume that the classifier is able to identify that a token is negatively correlated with the Hate Speech(1) class so it must know something about that token, namely, that it is strongly pulling towards the opposite class (because there are no other options) and this is much easier to imagine for 2 class problem.*